## Формирование выборки обучающих данных для задачи классификации (распознавание гистоновых меток)

In [20]:
import pandas as pd
import os
import numpy as np
import itertools
import random

In [34]:
chrs = ['chr'+str(n) for n in range (1,23)] + ['chrX', 'chrY', 'chrM']
#chrs = ['chr'+str(n) for n in range (1,4)]
print(chrs)

['chr1', 'chr2', 'chr3', 'chr4', 'chr5', 'chr6', 'chr7', 'chr8', 'chr9', 'chr10', 'chr11', 'chr12', 'chr13', 'chr14', 'chr15', 'chr16', 'chr17', 'chr18', 'chr19', 'chr20', 'chr21', 'chr22', 'chrX', 'chrY', 'chrM']


In [35]:
histone_map = "E001-H3K27me3.imputed.narrowPeak.bed.nPk"

In [36]:
data_h = pd.read_csv(histone_map, header=None, sep = '\t')

### Нормировка участков, соответствующих положениям гистоновых меток (центр+-100bp)

In [38]:
for i in range (0, len(data_h)):
    middle = int(round((data_h.iloc[i,2] + data_h.iloc[i,1])/2,0))
    data_h.iloc[i,1] = middle - 100
    data_h.iloc[i,2] = middle + 100

### Сборка участков sequence, соответствующих положениям гистоновых меток, в массивы

In [39]:
array_yes = []; array_no = []

for ch in chrs:
    chromFa =  "chromFa_hg19/%s.fa" % (ch)
    data_c = pd.read_csv(chromFa, sep = '\t')
    print(ch)

    string_chr = ''
    for i in range(0, len(data_c)):
        string_chr += data_c.iloc[i,0].upper()
    flag = 0

    for j in range (0, len(data_h)):
        if (data_h.iloc[j,0] == ch):
            start_yes = data_h.iloc[j,1]
            end_yes = data_h.iloc[j,2]
            el_yes = string_chr[start_yes:end_yes]
            if (flag == 0):
                start_no = 0
            else:
                start_no = data_h.iloc[j-1,2]
            flag = 1
            if 'N' not in el_yes:
                array_yes.append(el_yes)
                
            # Формируем участки по 200bp для array_no
            for k in range (start_no, start_yes, 200):
                if ((k+200) <= start_yes):
                    el_no = string_chr[k:k+200]
                    if (len(el_no)==200):
                        if 'N' not in el_no:
                            array_no.append(el_no)
        else:
            if (flag == 1):
                for k in range (end_yes, len(data_c)*50, 200):
                    if ((k+200) <= len(data_c)*50):
                        el_no = string_chr[k:k+200]
                        if (len(el_no)==200):
                            if 'N' not in el_no:
                                if el_no not in array_no:
                                    array_no.append(el_no)

chr1
chr2
chr3
chr4
chr5
chr6
chr7
chr8
chr9
chr10
chr11
chr12
chr13
chr14
chr15
chr16
chr17
chr18
chr19
chr20
chr21
chr22
chrX
chrY
chrM


In [40]:
while '' in array_yes:
    array_yes.remove('')
while '' in array_no:
    array_no.remove('')

### Делаем DataFrame с двумя классами

In [41]:
df_all = pd.DataFrame(columns = ['class', 'sequence'])

In [42]:
random.shuffle(array_no) # Чтобы примеры были из разных хромосом

for i in range (0, len(array_yes)):
    df_all.loc[i,'sequence'] = array_yes[i]
    df_all.loc[i,'class'] = 'yes'
for i in range (0, len(array_yes)):
    df_all.loc[i+len(array_yes),'sequence'] = array_no[i]
    df_all.loc[i+len(array_yes),'class'] = 'no'
print('okay')

okay


In [43]:
df_all = df_all.dropna()

In [44]:
file_df = open('dfshort.csv', 'w')
for i in range(0,len(df_all)):
    file_df.write(str(df_all.iloc[i,0]))
    file_df.write('\t')
    file_df.write(str(df_all.iloc[i,1]))
    file_df.write('\n')
file_df.close()

### One hot encoding

In [ ]:
df = pd.read_csv('df.csv', sep = '\t', header = None)
df = df.dropna()

In [ ]:
[[type(i) == str for i in df[1].values]]

In [ ]:
enc = {  'A':[1,0,0,0], 
         'C':[0,1,0,0], 
         'G':[0,0,1,0], 
         'T':[0,0,0,1]}

In [ ]:
df_encoded = np.array([[enc[char] for char in i] for i in df[1].values])

In [ ]:
df_encoded.shape

In [ ]:
X = df_encoded

In [ ]:
y = (df[0] == 'yes').values.astype(np.int)

### Counting di- and trinucleotide repeats

In [45]:
df = pd.read_csv('dfshort.csv', sep = '\t', header = None)
df = df.dropna()

In [46]:
[[type(i) == str for i in df[1].values]]

[[True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,


In [47]:
comb2 = list(itertools.product('ATCG', repeat=2))
comb2 = [i[0] + i[1] for i in comb2]
comb3 = list(itertools.product('ATCG', repeat=3))
comb3 = [i[0] + i[1] + i[2] for i in comb3]
combs = comb2 + comb3

In [48]:
df_comb = pd.DataFrame(columns = combs)

for n in range (0, len(df)):
    string = df.iloc[n,1]
    comb_dict = dict.fromkeys(combs, 0)
    for i in range (0, len(string)-1):
        try:
            a = string[i]+string[i+1]
            b = string[i]+string[i+1]+string[i+2]
            comb_dict[b]+=1
        except:
            a = string[i]+string[i+1]
        comb_dict[a]+=1
    df_comb.loc[n] = comb_dict

In [49]:
df_comb['class'] = df[0]

In [50]:
df_comb = df_comb.dropna()

In [ ]:
df_comb

In [33]:
file_df = open('df_RF.csv', 'w')
for i in range (0,len(df_comb)):
    for j in range (0,81):
        file_df.write(str(df_comb.iloc[i,j]))
        file_df.write('\t')
    file_df.write('\n')
file_df.close()